In [6]:
import pandas as pd 
import re

In [7]:
path = ("_data.csv")
df = pd.read_csv(path)
df = df[df["Адрес"].str.contains("Москва", case=False)]
pd.set_option('display.max_columns', None)
df.drop(["Unnamed: 0", "Тип", "Телефоны", "Описание", "Название ЖК", "Серия дома", "Ссылка на объявление"], inplace=True, axis=1)

In [8]:
# Меняем название колонок
rename_map = {'Количество комнат': "number_of_rooms", 'Метро': "metro", 'Адрес': "address", 'Площадь, м2': "apartment_area_m2", 'Дом': "house", 'Парковка': "parking_lot"}
df.rename(columns=rename_map, inplace=True)

In [9]:
# Заполняем пропущенные значения
df["number_of_rooms"].fillna("Апартаменты/свободная планировка", inplace=True)
df["metro"].fillna("не указана", inplace=True)
df["parking_lot"].fillna("нет", inplace=True)

df['Дополнительно'] = df['Дополнительно'].fillna('Нет данных')
df['Высота потолков, м'] = df['Высота потолков, м'].fillna(df['Высота потолков, м'].mean())
df['Лифт'] = df['Лифт'].fillna('Нет данных')
df['Мусоропровод'] = df['Мусоропровод'].fillna('Нет данных')
df['Можно с детьми/животными'] = df['Можно с детьми/животными'].fillna('Нет данных')
df.rename(columns={'Можно с детьми/животными': 'Children/pets allowed','Дополнительно': 'Additionally','Высота потолков, м': 'Ceiling_height',
                   'Лифт': 'Elevator','Мусоропровод': 'Garbage_chute'}, inplace= True)

# Переименовываем столбцы 'Цена', 'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел' в английские
df.rename(columns={'Цена': 'price', 'Ремонт': 'decor', 'Площадь комнат, м2': 'rooms_sq', 'Балкон': 'balcony', 'Окна': 'windows', 'Санузел': 'bathroom'}, inplace=True)

# Заполняем пустые значения столбца 'decor' (ремонт) значением 'decor_na'
df['decor'].fillna('decor_na', inplace=True)

# Создаем функцию подсчета суммы числовых значений в строке
def sum_numbers(string):
       numbers = re.findall(r'\d+', string)  # Используем регулярное выражение для поиска чисел
       numbers = [int(num) for num in numbers]  # Преобразуем найденные числа в целочисленный формат
       
       return sum(numbers)  # Вернем сумму чисел

# Заполняем непустые значения столбца 'rooms_sq' суммой числовых значения, используя функцию sum_numbers
df['rooms_sq'] = df['rooms_sq'].apply(lambda x: sum_numbers(x) if pd.notnull(x) else x)

# Заменяем 1 выброс средним значением.
df['rooms_sq'] = df['rooms_sq'].replace(79060990004, df['rooms_sq'].median())

# Заполняем пустые значения столбца 'rooms_sq' медианными значениями.
df['rooms_sq'].fillna(df['rooms_sq'].median(), inplace=True)

# Заполняем пустые значения столбца 'balcony' (Балкон) значением 'no_balcony'
df['balcony'].fillna('no_balcony',inplace=True)

# Заполняем пустые значения столбца 'windows' (Окна) значением 'windows_na'
df['windows'].fillna('windows_na',inplace=True)

# Заполняем пустые значения столбца 'bathroom' (Санузел) значением 'bathroom_na'
df['bathroom'].fillna('bathroom_na',inplace=True)

In [10]:
df.to_csv("data.csv")